<a href="https://colab.research.google.com/github/svobodn1y/PP_Matrix/blob/main/Digit_Recognizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Подключение и обработка данных

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import scipy

In [ ]:
data = pd.read_csv("train.csv")

In [ ]:
data.head()

In [ ]:
data = np.array(data)
m, n = data.shape
np.random.shuffle(data)

data_dev = data[0:1000].T # тестовая выборка
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255.0

data_train = data[1000:m].T # обучающая выборка
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.0

##Функции для обучения

In [ ]:
def init_params():
    W1 = np.random.normal(size=(10, 784)) * np.sqrt(1./784)
    b1 = np.random.normal(size=(10, 1)) * np.sqrt(1./10)
    W2 = np.random.normal(size=(10, 10)) * np.sqrt(1./20)
    b2 = np.random.normal(size=(10, 1)) * np.sqrt(1./784)
    return W1, b1, W2, b2

def ReLU(Z):
  return np.maximum(0, Z)

def softmax(Z):
    Z -= np.max(Z, axis=0)
    A = np.exp(Z) / np.sum(np.exp(Z), axis=0)
    return A

def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def ReLU_deriv(Z):
    return Z > 0

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size),Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    m = X.shape[1]
    one_hot_Y = one_hot(Y)
    dZ2 = 2 * (A2 - one_hot_Y)
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2, axis=1, keepdims=True)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1, axis=1, keepdims=True)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha *  db2
    return W1, b1, W2, b2

##Функции запуска и проверок

In [ ]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

def make_predictions(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_prop(W1, b1, W2, b2, X)
    predictions = get_predictions(A2)
    return predictions

def test_prediction(index, W1, b1, W2, b2):
    current_image = X_train[:, index, None]
    prediction = make_predictions(X_train[:, index, None], W1, b1, W2, b2)
    label = Y_train[index]
    print("Prediction: ", prediction)
    print("Label: ", label)

    current_image = current_image.reshape((28, 28)) * 255
    plt.gray()
    plt.imshow(current_image, interpolation='nearest')
    plt.show()

##Обучение модели на _train

In [ ]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.09, 500)

Iteration:  0
0.11202439024390244
Iteration:  10
0.4949512195121951
Iteration:  20
0.6245365853658537
Iteration:  30
0.7086097560975609
Iteration:  40
0.7670731707317073
Iteration:  50
0.7974146341463415
Iteration:  60
0.8163170731707317
Iteration:  70
0.8312439024390244
Iteration:  80
0.8430975609756097
Iteration:  90
0.8526341463414634
Iteration:  100
0.8604634146341463
Iteration:  110
0.8664634146341463
Iteration:  120
0.8725121951219512
Iteration:  130
0.8772439024390244
Iteration:  140
0.8814634146341463
Iteration:  150
0.8839512195121951
Iteration:  160
0.8869024390243903
Iteration:  170
0.8893658536585366
Iteration:  180
0.8914390243902439
Iteration:  190
0.8929756097560976
Iteration:  200
0.8949268292682927
Iteration:  210
0.8960731707317073
Iteration:  220
0.897219512195122
Iteration:  230
0.8985365853658537
Iteration:  240
0.8995853658536586
Iteration:  250
0.9007317073170732
Iteration:  260
0.9017317073170732
Iteration:  270
0.9027560975609756
Iteration:  280
0.9035121951219

##Проверка модели на _dev

In [ ]:
dev_predictions = make_predictions(X_dev, W1, b1, W2, b2)
get_accuracy(dev_predictions, Y_dev)

0.916

In [ ]:
n = 10
index_list = np.random.randint(0,40000, n)
for i in index_list:
  test_prediction(i, W1, b1, W2, b2)

##Финальное обучение на всех данных и предикт на _test

In [ ]:
data_TRAIN = pd.read_csv("train.csv")
data_TRAIN = np.array(data_TRAIN)
m, n = data.shape

np.random.shuffle(data_TRAIN)
data_TRAIN = data_TRAIN.T

Y = data_TRAIN[0]
X = data_TRAIN[1:n]
X = X / 255.0

def gradient_descent_without_logs(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
    return W1, b1, W2, b2

W1, b1, W2, b2 = gradient_descent_without_logs(X, Y, 0.09, 500)

In [ ]:
data_test = pd.read_csv("test.csv")
data_test = np.array(data_test).T

dev_predictions = make_predictions(data_test, W1, b1, W2, b2)

dev_predictions


In [ ]:
df = pd.DataFrame({
    'ImageId': np.arange(1, len(dev_predictions) + 1),
    'Label': dev_predictions})

df.to_csv("out.csv", header=True, index=False)

df